In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Import BTC-USD.csv
btc_file="Resources/BTC-USD.csv"
btc_usd_df=pd.read_csv(btc_file)
btc_usd_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,9/17/2014,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,9/18/2014,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,9/19/2014,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,9/20/2014,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,9/21/2014,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [3]:
#Import ETH-USD.csv
eth_file="Resources/ETH-USD.csv"
eth_usd_df=pd.read_csv(eth_file)
eth_usd_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08
1,11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08
2,11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08
3,11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09
4,11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09


In [4]:
# Renamed BTC columns
btc_rename=btc_usd_df.rename (columns={"Open":"BTC Open",
                                      "High":"BTC High",
                                      "Low":"BTC Low",
                                      "Close":"BTC Close",
                                      "Adj Close":"BTC Adj Close",
                                      "Volume":"BTC Volume"})
btc_rename.head()

,Date,BTC Open,BTC High,BTC Low,BTC Close,BTC Adj Close,BTC Volume
0,9/17/2014,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,9/18/2014,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,9/19/2014,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,9/20/2014,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,9/21/2014,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [5]:
# Renamed ETH columns
eth_rename=eth_usd_df.rename (columns={"Open":"ETH Open",
                                      "High":"ETH High",
                                      "Low":"ETH Low",
                                      "Close":"ETH Close",
                                      "Adj Close":"ETH Adj Close",
                                      "Volume":"ETH Volume"})
eth_rename.head()

,Date,ETH Open,ETH High,ETH Low,ETH Close,ETH Adj Close,ETH Volume
0,11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08
1,11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08
2,11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08
3,11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09
4,11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09


In [6]:
# Setting the Date column as index
btc_rename=btc_rename.set_index(["Date"])
btc_rename.head()

,BTC Open,BTC High,BTC Low,BTC Close,BTC Adj Close,BTC Volume
Date,,,,,,
9/17/2014,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
9/18/2014,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
9/19/2014,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
9/20/2014,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
9/21/2014,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [7]:
# Setting the Date column as index
eth_rename=eth_rename.set_index(["Date"])
eth_rename.head()

,ETH Open,ETH High,ETH Low,ETH Close,ETH Adj Close,ETH Volume
Date,,,,,,
11/9/2017,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08
11/10/2017,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08
11/11/2017,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08
11/12/2017,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09
11/13/2017,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09


In [8]:
#Joined the Dataframes on the Date
combined_df=btc_rename.merge(eth_rename,left_index=True,right_index=True)
combined_df.head()

,BTC Open,BTC High,BTC Low,BTC Close,BTC Adj Close,BTC Volume,ETH Open,ETH High,ETH Low,ETH Close,ETH Adj Close,ETH Volume
Date,,,,,,,,,,,,
11/9/2017,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,308.644989,329.451996,307.056000,320.884003,320.884003,8.932500e+08
11/10/2017,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,320.670990,324.717987,294.541992,299.252991,299.252991,8.859860e+08
11/11/2017,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,298.585999,319.453003,298.191986,314.681000,314.681000,8.423010e+08
11/12/2017,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,314.690002,319.153015,298.513000,307.907990,307.907990,1.613480e+09
11/13/2017,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,307.024994,328.415009,307.024994,316.716003,316.716003,1.041890e+09


In [9]:
# Verified the data types in the dataframe
combined_df.dtypes

BTC Open         float64
BTC High         float64
BTC Low          float64
BTC Close        float64
BTC Adj Close    float64
BTC Volume       float64
ETH Open         float64
ETH High         float64
ETH Low          float64
ETH Close        float64
ETH Adj Close    float64
ETH Volume       float64
dtype: object

In [ ]:
protocol = 'postgresql'
username = '<user name>'
password = '<password>'
host = 'localhost'
port = 5432
database_name = 'customer_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)